In [6]:
import tensorflow as tf
import numpy as np

### Tensors

 Tensors are multidimensional arrays

In [28]:
x = tf.constant([[1, 2, 4],
                 [3, 5, 6]])

x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [29]:
print(f"Tensor x \n {x}")
print(f"Tensor x shape :   {x.shape}")
print(f"Tensor x dtype :   {x.dtype}")

Tensor x 
 [[1 2 4]
 [3 5 6]]
Tensor x shape :   (2, 3)
Tensor x dtype :   <dtype: 'int32'>


In [30]:
y = tf.convert_to_tensor(np.array([[1, 0, 1],[1, 1, 1]]))
y

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 0, 1],
       [1, 1, 1]])>

In [31]:
### Tensor operations

print(f"Transpose of x \n {x @ tf.transpose(x)}")

print(f"Concat x & y \n {tf.concat([x,y], axis=0)}")

print(f"Reduce sum {tf.reduce_sum([1,2,3])}")

Transpose of x 
 [[21 37]
 [37 70]]
Concat x & y 
 [[1 2 4]
 [3 5 6]
 [1 0 1]
 [1 1 1]]
Reduce sum 6


### Variables

Normal tf.Tensor objects are immutable. To store model weights (or other mutable state) in TensorFlow use a tf.Variable.

In [37]:
x_var = tf.Variable(x)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [33]:
### Adding values to varible within the same ref
x_var.assign_add(y)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[2, 2, 5],
       [4, 6, 7]])>

In [38]:
x_var.assign([[1, 2, 4],
       [3, 5, 6]])
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [39]:
#Most tensor operations work on variables as expected, although variables cannot be reshaped. 
## its creating tensor not varible

x_var_re = tf.reshape(x_var, [3,2])
x_var_re

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [4, 3],
       [5, 6]])>

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'